# 2 Step One-vs-All Prediction
Score on Driven Data: 0.7879

**Summary:** In our research, we found out that there is a strong seperation between non-functional and the two functional classes (functional & functional needs repair). Therefore, this approach starts with a one-vs-all classification to find if the pump is non-functional or not. In the second step, a different model tries to differentiate those pump predicted 'functional', to find if they are 'functional' or 'functional needs repair'. 

**Content:**
1. Data Loading
2. Data Cleaning
3. One-hot Encoding and Scaling
4. Feature Creation
    1. Naive Bayes
    2. Linear Discriminant Analysis & Logistic Regression
    3. Principal Component Analysis & kNN
5. Feature Selection
6. Model Step 1
7. Model Step 2
8. Prediction

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import PowerTransformer
from sklearn.inspection import permutation_importance
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier

import xgboost as xgb

%matplotlib inline

## 1. Data Loading
In our exploratory data analysis, we have identified a number of columns which are clearly irrelevant, that contain duplicate information or that are categorical and contain too many categories to use them. We drop these columns directly after loading the data.

In [2]:
# load data
X_train = pd.read_csv('train_features.csv')
y_train = pd.read_csv('train_labels.csv')
X_test = pd.read_csv('test_features.csv')
y_test = pd.read_csv('submission_format.csv')

# merge features and labels on train set
train = X_train.copy()
train = train.merge(y_train, how = 'left', on = 'id')

In [3]:
# column to always drop
columns_to_drop = [
    'id',
    'subvillage',
    'region_code',
    'district_code',
    'wpt_name',
    'recorded_by',
    'scheme_name',
    'management_group',
    'payment',
    'extraction_type_group',
    'extraction_type_class',
    'waterpoint_type_group',
    'quality_group',
    'quantity_group',
    'source_type',
    'source_class',
    'num_private', 
    'date_recorded',
    'funder',
    'installer',
    'lga',
    'ward',
    'scheme_management'
]

In [4]:
# drop columns
X_train.drop(columns_to_drop, axis = 1, inplace = True)
X_test.drop(columns_to_drop, axis = 1, inplace = True)

In [5]:
# show remaining columns
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 17 columns):
amount_tsh           59400 non-null float64
gps_height           59400 non-null int64
longitude            59400 non-null float64
latitude             59400 non-null float64
basin                59400 non-null object
region               59400 non-null object
population           59400 non-null int64
public_meeting       56066 non-null object
permit               56344 non-null object
construction_year    59400 non-null int64
extraction_type      59400 non-null object
management           59400 non-null object
payment_type         59400 non-null object
water_quality        59400 non-null object
quantity             59400 non-null object
source               59400 non-null object
waterpoint_type      59400 non-null object
dtypes: float64(3), int64(3), object(11)
memory usage: 7.7+ MB


## 2. Data Cleaning
**construction_year:** This feature has a large portion of values which are 0. To clean this data, we impute these missing values with the mean construction year. Also, we create a new column to keep the information of whether the construction year was properly recorded or not. 

**longitude, latitude:** The two geolocation columns also have values which are zero. To impute these values, we impute them with the mean longitude and latitude for the region that the pump is located in. Again, we create a column to store the information of whether the longitude and latitude was recorded or not. 

**public_meeting, permit:** These two boolean variables contain a few missing values (around 3000, ~5% of the data). We decided to impute them with the majority class. 

In [6]:
# create a column storing the info whether construction year was recorded or not
X_train['construction_year_recorded'] = np.where(X_train.construction_year == 0, False, True)
X_test['construction_year_recorded'] = np.where(X_test.construction_year == 0, False, True)

# replace construction_year == 0 with the mean construction year
mean_construction_year = round(X_train.loc[X_train.construction_year != 0, 'construction_year'].mean(), 0)
X_train.loc[X_train.construction_year == 0, 'construction_year'] = mean_construction_year
X_test.loc[X_test.construction_year == 0, 'construction_year'] = mean_construction_year

In [7]:
# create a column storing the info whether longitude/latitude was recorded or not
X_train['longitude_recorded'] = np.where(abs(X_train.longitude) < 0.1, False, True)
X_train['latitude_recorded'] = np.where(abs(X_train.latitude) < 0.1, False, True)

X_test['longitude_recorded'] = np.where(X_test.longitude < 0.1, False, True)
X_test['latitude_recorded'] = np.where(X_test.latitude < 0.1, False, True)

# calculate the mean longitude/latitude for each region
mean_longitude = [X_train.loc[X_train.region == region,'longitude'].mean() for region in X_train.region.unique()]
mean_latitude = [X_train.loc[X_train.region == region,'latitude'].mean() for region in X_train.region.unique()]

mean_location = pd.DataFrame(data = {'mean_longitude' : mean_longitude,
                                     'mean_latitude' : mean_latitude},
                             index = X_train.region.unique())

# replace longitudes/latitudes close to 0 with the mean longitude/latitude for the region
for i in range(0,X_train.shape[0]):
    
    # replace longitudes around 0 with the mean for the respective region of the observation
    if abs(X_train.loc[i, 'longitude']) < 0.1:
        X_train.loc[i, 'longitude'] = mean_location.loc[X_train.loc[i, 'region'], 'mean_longitude']
        
    # do the same for the latitude
    if abs(X_train.loc[i, 'latitude']) < 0.1:
        X_train.loc[i, 'latitude'] = mean_location.loc[X_train.loc[i, 'region'], 'mean_latitude']

# same for test set
for i in range(0,X_test.shape[0]):
    
    # replace longitudes around 0 with the mean for the respective region of the observation
    if abs(X_test.loc[i, 'longitude']) < 0.1:
        X_test.loc[i, 'longitude'] = mean_location.loc[X_test.loc[i, 'region'], 'mean_longitude']
        
    # do the same for the latitude
    if abs(X_test.loc[i, 'latitude']) < 0.1:
        X_test.loc[i, 'latitude'] = mean_location.loc[X_test.loc[i, 'region'], 'mean_latitude']

In [8]:
# replace missing values in public_meeting with the majority category (True)
X_train.loc[X_train.public_meeting.isna(), 'public_meeting'] = True
X_test.loc[X_test.public_meeting.isna(), 'public_meeting'] = True

# replace missing values in permit with the majority category (True)
X_train.loc[X_train.permit.isna(), 'permit'] = True
X_test.loc[X_test.permit.isna(), 'permit'] = True

## 3. One-hot Encoding and Scaling
All the algorithms we use below (LDA, PCA, Naive Bayes and XGBoost) require numerical data. Therefore, we one-hot encode the categorical data and scale the numerical data using a power transformer. 

We also map the target to be numerical and create two binary target variables, one for step1 and one for step2.

In [9]:
# one-hot encoding
X_train = pd.get_dummies(X_train, 
                         prefix = X_train.select_dtypes('object').columns, 
                         columns = X_train.select_dtypes('object').columns,
                         drop_first = False
                        )

X_test = pd.get_dummies(X_test, 
                         prefix = X_test.select_dtypes('object').columns, 
                         columns = X_test.select_dtypes('object').columns,
                         drop_first = False
                        )

# power transformation of numerical columns
numerical_columns = X_train.select_dtypes(['int64', 'float64']).columns

pt = PowerTransformer()
X_train.loc[:,numerical_columns] = pt.fit_transform(X_train.loc[:,numerical_columns])
X_test.loc[:,numerical_columns] = pt.transform(X_test.loc[:,numerical_columns])

# add columns to test set that only exist in train set
X_test[list(set(X_train.columns).difference(set(X_test.columns)))[0]] = 0

# make sure columns are in the same order
X_train = X_train[sorted(X_train.columns)].copy()
X_test = X_test[sorted(X_test.columns)].copy()

# convert boolean into numerical
for column in X_train.select_dtypes('bool').columns:
    X_train[column] = X_train[column].astype(int)
    X_test[column] = X_test[column].astype(int)

C:\Users\danie\Anaconda3\lib\site-packages\numpy\core\_methods.py:121: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
C:\Users\danie\Anaconda3\lib\site-packages\numpy\core\_methods.py:122: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims)


In [10]:
# create a mapping for the multinomial classes
multinomial_classes = {
    'functional' : 0,
    'non functional' : 1,
    'functional needs repair' : 2
}

# create the inverse mapping
classes_inv = {v: k for k, v in multinomial_classes.items()}

# map the target to numerical
y_train_multinomial = y_train.status_group.map(multinomial_classes).copy()

# create binary classes for both steps
y_train_binary_step1 = np.where(y_train_multinomial == 1, 1, 0)
y_train_binary_step2 = y_train_multinomial[y_train_multinomial != 1].copy()

In [11]:
# split X_train for step1 and step2
X_train_step1 = X_train.copy()
X_train_step2 = X_train.loc[y_train_multinomial != 1,:].copy()

## 4. Feature Creation
To add more information to the dataset, we use different classifiers to create a new feature with the probability that an each observation is of class 0 or class 1. We use three approaches here:

**Naive Bayes:** We train a naive bayes classifier on only the one-hot encoded categorical features. It's accuracy is 0.76.

**LDA & Logistic Regression:** We first use a linear discriminant analysis (LDA) to get the dimension of biggest variance between the two classes in step1. Since LDA doesn't allow us to predict probabilities, we train a logistic regression on that single feature. It's accuracy is 0.79.

**PCA & kNN:** We use PCA with the first 10 principal components to create a new feature space with a reduced number of features. This allows us to train a kNN model without having the curse of dimensionality that we would have with the over 100 dimensions of the train set. We use 21 neighbors, because our research found the the number of neighbors doesn't have a large impact on predictive power and with a larger number of neighbors we have lower variance and it allows use to get more distint probabilities when using the predict_proba function. This kNN model has an accuracy of 0.82.

We only used this approach for step1, because we saw that for step2, these models didn't have any predictive power. In step2, the class ratio is 0.88 to 0.12 ('functional' to 'functional needs repair'), and all these models didn't score higher than 0.88.

### A. Naive Bayes

In [12]:
# take only the one-hot encoded categorical features
nb_features = X_train_step1.select_dtypes('uint8').columns

# create and fit a naive bayes classifier for step1, print cross-validation score for reference
nb_step1 = GaussianNB()

step1_cross_val_score = cross_val_score(nb_step1, X_train_step1[nb_features], y_train_binary_step1)
print(f'Naive Bayes step1 cross-validation: {step1_cross_val_score}')

nb_step1.fit(X_train_step1[nb_features], y_train_binary_step1)

# predict probabilities
X_train_step1['naive_bayes_proba'] = nb_step1.predict_proba(X_train_step1[nb_features])[:,1]

Naive Bayes step1 cross-validation: [0.76212121 0.75294613 0.76498316 0.76077441 0.76599327]


### B. Linear Discriminant Analysis & Logistic Regression

In [13]:
# create and fit an LDA model
lda_step1 = LinearDiscriminantAnalysis(solver = 'svd')
lda_step1.fit(X_train_step1, y_train_binary_step1)

# transform train set
lda_train_step1 = lda_step1.transform(X_train_step1)

# create and fit a logistic regression model, print cross-validation score for reference
logit_step1 = LogisticRegression(penalty = 'none', solver = 'lbfgs')

step1_cross_val_score = cross_val_score(logit_step1, lda_train_step1, y_train_binary_step1)
print(f'LDA & Logit Model step1 cross-validation: {step1_cross_val_score}')

logit_step1.fit(lda_train_step1, y_train_binary_step1)

# predict probabilities
X_train_step1['logit_proba'] = logit_step1.predict_proba(lda_train_step1)[:,1]

LDA & Logit Model step1 cross-validation: [0.79267677 0.78905724 0.78897306 0.78728956 0.7989899 ]


### Principal Component Analysis & kNN

In [14]:
# create and fit a PCA model
pca_step1 = PCA(n_components = 10)
pca_step1.fit(X_train_step1)

# transform data
pca_train_step1 = pca_step1.transform(X_train_step1)

# create and fit a kNN model
knn_step1 = KNeighborsClassifier(n_neighbors = 21)

step1_cross_val_score = cross_val_score(knn_step1, pca_train_step1, y_train_binary_step1)
print(f'PCA & kNN Model step1 cross-validation: {step1_cross_val_score}')

knn_step1.fit(pca_train_step1, y_train_binary_step1)

# predict probabilities
X_train_step1['knn_proba'] = knn_step1.predict_proba(pca_train_step1)[:,1]

PCA & kNN Model step1 cross-validation: [0.82626263 0.81801347 0.81675084 0.81582492 0.81700337]


## 5. Feature Selection
For feature selection, we use a method called **permutation importance**. In this method, we train a classifier on a train set. Then the permutation importance algorithm makes predictions on a validation set for a base model. Then, it makes predictions again, this time with the i-th column being randomized . If this randomization of the i-th column leads to a drop in accuracy score, then we know that this feature is relevant. We decide to keep all features where the randomization leads to a drop in accuracy of at least 0.001.

In [15]:
# create a train-validation split for feature selection
X_train_step1_split, X_validation_step1_split, y_train_binary_step1_split, y_validation_binary_step1_split \
    = train_test_split(X_train_step1,
                       y_train_binary_step1,
                       test_size = 0.2,
                       stratify = y_train_binary_step1,
                       random_state = 12)

# create xgboost model
xgb_model = xgb.XGBClassifier(max_depth = 12,
                              n_estimators = 100,
                              learning_rate = 0.1,
                              gamma = 0.001,
                              booster = 'gbtree',
                              min_child_weight = 20,
                              subsample = 0.8, 
                              colsample_bytree = 0.8,
                              colsample_bylevel = 0.8,
                              colsample_bynode = 0.8,
                              scale_pos_weight = 1,
                              seed = 27)

xgb_model.fit(X_train_step1_split, y_train_binary_step1_split)

# calculate permutation_importance
permutation_importance_step1 = permutation_importance(xgb_model, 
                                                      X_validation_step1_split, 
                                                      y_validation_binary_step1_split,
                                                      n_repeats = 20,
                                                      n_jobs = -1,
                                                      random_state = 62)

permutation_importance_df_step1 = pd.DataFrame({'feature' : X_train_step1_split.columns,
                                                'permutation_importance' : permutation_importance_step1['importances_mean']})

permutation_importance_df_step1.sort_values(by = ['permutation_importance'], ascending = False, inplace = True)

# select those features with a permutation importance > 0.001
relevant_features_step1 = permutation_importance_df_step1.loc[permutation_importance_df_step1.permutation_importance > 0.001, 
                                                              'feature']
print(relevant_features_step1)

# drop irrelevant features
X_train_step1 = X_train_step1[relevant_features_step1].copy()

# remove splitted data as it's not needed anymore
del X_train_step1_split, X_validation_step1_split, y_train_binary_step1_split, y_validation_binary_step1_split

110                knn_proba
109              logit_proba
108        naive_bayes_proba
33                 longitude
31                  latitude
57              quantity_dry
107    waterpoint_type_other
30                gps_height
Name: feature, dtype: object


In [16]:
# create a train-validation split for feature selection
X_train_step2_split, X_validation_step2_split, y_train_binary_step2_split, y_validation_binary_step2_split \
    = train_test_split(X_train_step2,
                       y_train_binary_step2,
                       test_size = 0.2,
                       stratify = y_train_binary_step2,
                       random_state = 12)

# create xgboost model
xgb_model = xgb.XGBClassifier(max_depth = 12,
                              n_estimators = 100,
                              learning_rate = 0.1,
                              gamma = 0.001,
                              booster = 'gbtree',
                              min_child_weight = 20,
                              subsample = 0.8, 
                              colsample_bytree = 0.8,
                              colsample_bylevel = 0.8,
                              colsample_bynode = 0.8,
                              scale_pos_weight = 1,
                              seed = 27)

xgb_model.fit(X_train_step2_split, y_train_binary_step2_split)

# calculate permutation_importance
permutation_importance_step2 = permutation_importance(xgb_model, 
                                                      X_validation_step2_split, 
                                                      y_validation_binary_step2_split,
                                                      n_repeats = 20,
                                                      n_jobs = -1,
                                                      random_state = 62)

permutation_importance_df_step2 = pd.DataFrame({'feature' : X_train_step2_split.columns,
                                                'permutation_importance' : permutation_importance_step2['importances_mean']})

permutation_importance_df_step2.sort_values(by = ['permutation_importance'], ascending = False, inplace = True)

# select those features with a permutation importance > 0.001
relevant_features_step2 = permutation_importance_df_step2.loc[permutation_importance_df_step2.permutation_importance > 0.001, 
                                                              'feature']
print(relevant_features_step2)

# drop irrelevant features
X_train_step2 = X_train_step2[relevant_features_step2].copy()

# remove splitted data as it's not needed anymore
del X_train_step2_split, X_validation_step2_split, y_train_binary_step2_split, y_validation_binary_step2_split

33                                       longitude
103    waterpoint_type_communal standpipe multiple
31                                        latitude
10                               construction_year
15                         extraction_type_gravity
105                      waterpoint_type_hand pump
3                                 basin_Lake Rukwa
58                                 quantity_enough
56                                  public_meeting
53                            payment_type_unknown
107                          waterpoint_type_other
30                                      gps_height
49                          payment_type_never pay
65                                   region_Iringa
Name: feature, dtype: object


## 6. Model Step 1
Since XGBoost takes a long time to fit, we can only do a grid-search on a few parameters. However, playing around with parameters in other notebooks has shown that max_depth and min_child_weight are the most impactful to both increase model performance and to avoid overfitting. 

Decreasing the 'learning_rate' makes the model perform worse. Changing 'gamma' has had absolutely no impact on model performance. Decreasing 'subsample' reduces model overfitting, but also at a large expense to test score. 

In step 1, we predict if an observation is 'functional', 'functional needs repair' (class 0) or 'non_functional' (class 1).

In [17]:
# define parameter grid
params = {
    'max_depth' : [10, 12],
    'min_child_weight' : [20, 40]
}

# create xgboost model
xgb_model = xgb.XGBClassifier(learning_rate = 0.1,
                              gamma = 0.001,
                              subsample = 0.8, 
                              colsample_bytree = 0.8,
                              colsample_bylevel = 0.8,
                              colsample_bynode = 0.8,
                              scale_pos_weight = 1,
                              seed = 27)

# create grid search object
grid_xgb_step1 = GridSearchCV(estimator = xgb_model, 
                              param_grid = params, 
                              scoring='accuracy',
                              n_jobs=-1,
                              cv=5,
                              refit = True,
                              return_train_score = True,
                              verbose = 1)

# fit the model
grid_xgb_step1.fit(X_train_step1, y_train_binary_step1)

# read results of grid search into dataframe
cv_results_df_step1 = pd.DataFrame(grid_xgb_step1.cv_results_)

# print results
cv_results_df_step1[['params', 'mean_train_score', 'mean_test_score']].sort_values(by = ['mean_test_score'], ascending = False)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   44.3s finished


,params,mean_train_score,mean_test_score
2,"{'max_depth': 12, 'min_child_weight': 20}",0.872934,0.848805
0,"{'max_depth': 10, 'min_child_weight': 20}",0.867551,0.848569
3,"{'max_depth': 12, 'min_child_weight': 40}",0.864268,0.847576
1,"{'max_depth': 10, 'min_child_weight': 40}",0.861090,0.846970


## 7. Model Step 2
In this step, out of all observations that were classified class 0 ('functional' or 'functional needs repair') are no classified to be 'functional' (class 0 in step 2) or 'functional needs repair' (class 2 in step 2).

In [18]:
# define parameter grid
params = {
    'max_depth' : [10, 12],
    'min_child_weight' : [20, 40]
}

# create xgboost model
xgb_model = xgb.XGBClassifier(learning_rate = 0.1,
                              gamma = 0.001,
                              subsample = 0.8, 
                              colsample_bytree = 0.8,
                              colsample_bylevel = 0.8,
                              colsample_bynode = 0.8,
                              scale_pos_weight = 1,
                              seed = 27)

# create grid search object
grid_xgb_step2 = GridSearchCV(estimator = xgb_model, 
                              param_grid = params, 
                              scoring='accuracy',
                              n_jobs=-1,
                              cv=5,
                              refit = True,
                              return_train_score = True,
                              verbose = 1)

# fit the model
grid_xgb_step2.fit(X_train_step2, y_train_binary_step2)

# read results of grid search into dataframe
cv_results_df_step2 = pd.DataFrame(grid_xgb_step2.cv_results_)

# print results
cv_results_df_step2[['params', 'mean_train_score', 'mean_test_score']].sort_values(by = ['mean_test_score'], ascending = False)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   33.8s finished


,params,mean_train_score,mean_test_score
2,"{'max_depth': 12, 'min_child_weight': 20}",0.907221,0.900044
0,"{'max_depth': 10, 'min_child_weight': 20}",0.905519,0.899634
3,"{'max_depth': 12, 'min_child_weight': 40}",0.901999,0.898075
1,"{'max_depth': 10, 'min_child_weight': 40}",0.900898,0.897802


## 8. Prediction
We apply all feature creation here to the test set and the run the two models trained above. After that we map back to the alphanumerical description of the classes (0 -> 'functional', 1 -> 'non_functional', 2 -> 'functional needs repair'), as per the requirements of the submission file.

In [19]:
# add new features to test set
X_test['naive_bayes_proba'] = nb_step1.predict_proba(X_test[nb_features])[:,1]

lda_test = lda_step1.transform(X_test)
X_test['logit_proba'] = logit_step1.predict_proba(lda_test)[:,1]

pca_test = pca_step1.transform(X_test)
X_test['knn_proba'] = knn_step1.predict_proba(pca_test)[:,1]

# Step 1
y_pred_step1 = grid_xgb_step1.predict(X_test[relevant_features_step1])

# Step 2
y_pred_step2 = grid_xgb_step2.predict(X_test.loc[y_pred_step1 == 0, relevant_features_step2])

# combine results
y_pred = y_pred_step1.copy()
y_pred[y_pred_step1 == 0] = y_pred_step2

# map back to string classes
y_pred = pd.Series(y_pred).map(classes_inv)

# create submission data frame
y_test.loc[:,'status_group'] = y_pred

# write to csv
y_test.to_csv('submission10.csv', index = False)

In [20]:
# check counts of predicted classes to make sure the predictions make sense
y_test.status_group.value_counts()

functional                 9475
non functional             5036
functional needs repair     339
Name: status_group, dtype: int64